# Step1: Installing Required Libraries

In [1]:
%pip install --upgrade pip --quiet
%pip install --upgrade datasets --quiet

%pip install \
    torch \
    torchdata \
    transformers \
    evaluate \
    rouge_score \
    loralib \
    peft --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Step2: Import Libraries

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

# Step3: Load Dataset and Analyze

In [3]:
# Load Data
dataset_name = 'knkarthick/dialogsum'
dataset = load_dataset(path=dataset_name)

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


Observations - Count of data
1) Train Data - 12460
2) Validation Data - 500
3) Test Data - 1500


In [5]:
dash_line = '-'.join('' for i in range(100))
dash_line

'---------------------------------------------------------------------------------------------------'

In [6]:
# Checking the examples

example_indices = [50, 200]

for i, idx in enumerate(example_indices, start=1):
    print(f'{dash_line}\nExample: {i}\n{dash_line}')
    print('INPUT DIALOGUE:')
    print(dataset['test'][idx]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][idx]['summary'])
    print(f'{dash_line}\n')


---------------------------------------------------------------------------------------------------
Example: 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: Yeah. Just pull on this strip. Then peel off the back.
#Person2#: You might make a few enemies this way.
#Person1#: If they don't think this is fun, they're not meant to be our friends.
#Person2#: You mean your friends. I think it's cruel.
#Person1#: Yeah. But it's fun. Look at those two ugly old ladies. . . or are they men?
#Person2#: Hurry! Get a shot!. . . Hand it over!
#Person1#: I knew you'd come around. . .
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is about to make a prank. #Person2# thinks it's cruel at first but then joins.
---------------------------------------------------------------------------------------------------

-------------------------------

# Step3: Load Model adn Tokenizer

In [7]:
# Load Tokenizer
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [9]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [8]:
print(tokenizer)

T5TokenizerFast(name_or_path='google/flan-t5-base', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>'

In [12]:
sentence = "What time is it, Ajit?"

sentence_encoded = tokenizer.encode(text=sentence, return_tensors='pt')

print(sentence_encoded)

tensor([[  363,    97,    19,    34,     6,    71, 13272,    58,     1]])


In [15]:
sentence_decoded = tokenizer.decode(sentence_encoded[0], skip_special_tokens=True)

print(sentence_decoded)

What time is it, Ajit?


In [23]:
sentence_model_output = model.generate(inputs=sentence_encoded, max_new_tokens=100)

print(sentence_model_output)

tensor([[    0, 29415,  3246,     1]])


In [21]:
sentence_model_decoded = tokenizer.decode(sentence_model_output[0], skip_special_tokens=True)

print(sentence_model_decoded)

4:00 PM


# Step4: Model Output Without Prompt Engineering

In [16]:
print(dataset['test'][1]['dialogue'])

#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with the memo. Wh

In [17]:
print(dataset['test'][1]['summary'])

In order to prevent employees from wasting time on Instant Message programs, #Person1# decides to terminate the use of those programs and asks Ms. Dawson to send out a memo to all employees by the afternoon.


In [26]:
example_indices = [50, 200]

for i, idx in enumerate(example_indices):
    dialogue = dataset['test'][idx]['dialogue']
    summary = dataset['test'][idx]['summary']

    dialogue_encoded = tokenizer.encode(text=dialogue, return_tensors='pt')
    model_output = model.generate(inputs=dialogue_encoded, max_new_tokens=100)
    summary_decoded = tokenizer.decode(token_ids=model_output[0], skip_special_tokens=True)

    print(f'{dash_line}\nExample: {i}\n{dash_line}')
    print(f'INPUT DIALOGUE:\n{dialogue}\n{dash_line}')
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n{dash_line}')
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{summary_decoded}\n')

---------------------------------------------------------------------------------------------------
Example: 0
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: Yeah. Just pull on this strip. Then peel off the back.
#Person2#: You might make a few enemies this way.
#Person1#: If they don't think this is fun, they're not meant to be our friends.
#Person2#: You mean your friends. I think it's cruel.
#Person1#: Yeah. But it's fun. Look at those two ugly old ladies. . . or are they men?
#Person2#: Hurry! Get a shot!. . . Hand it over!
#Person1#: I knew you'd come around. . .
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is about to make a prank. #Person2# thinks it's cruel at first but then joins.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMP

# Step 5: Prompting - Summarize with an Instruction Prompt

In [59]:
def make_zero_shot_prompt(example_indices):
    idx = example_indices[0]
    dialogue = dataset['test'][idx]['dialogue']
    summary = dataset['test'][idx]['summary']

    prompt = f"""
    Summarize the following conversation.
    
    {dialogue}
    
    Summary:
    """
    return prompt

1)Context Length is 512 tokens

In [60]:
def make_few_shot_prompt(example_indices_full, example_index_to_summarize, question):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
        Dialogue:
        
        {dialogue}
        
        {question}
        {summary}
        
        """
        
        dialogue = dataset['test'][example_index_to_summarize]['dialogue']
        
        prompt += f"""
        
        Dialogue:
        
        {dialogue}
        {question}
        """
        
    return prompt

In [61]:
def generate_summary(example_index_to_summarize, prompt):
    summary = dataset['test'][example_index_to_summarize]['summary']
    
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    model_output = model.generate(inputs=inputs, max_new_tokens=100)
    output = tokenizer.decode(token_ids=model_output[0], skip_special_tokens=True)

    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION:\n{output}')


## 1) Zero Shot

In [62]:
example_indices = [200]


zero_shot_prompt = make_zero_shot_prompt(example_indices)

print(zero_shot_prompt)

generate_summary(example_indices, zero_shot_prompt)


    Summarize the following conversation.
    
    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
    
    Summary:
    


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
["#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."]

---------------------------------------------------------------------------------------------------
MODEL GENERATION:
#Person1#: I'm thinking of upgrading my computer.


## 1) One Shot

In [66]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_few_shot_prompt(example_indices_full, example_index_to_summarize, "What conversation is going on?")

print(one_shot_prompt)

generate_summary(example_index_to_summarize, one_shot_prompt)


        Dialogue:
        
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
        
        What conversation is going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        
        
        Dialogue:
        
        #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Per

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION:
#Person1: I'm thinking of upgrading my computer.


In [68]:
example_indices_full = [100]
example_index_to_summarize = 200

one_shot_prompt = make_few_shot_prompt(example_indices_full, example_index_to_summarize, "What conversation is going on?")

print(one_shot_prompt)

generate_summary(example_index_to_summarize, one_shot_prompt)


        Dialogue:
        
        #Person1#: OK, that's a cut! Let's start from the beginning, everyone.
#Person2#: What was the problem that time?
#Person1#: The feeling was all wrong, Mike. She is telling you that she doesn't want to see you any more, but I want to get more anger from you. You're acting hurt and sad, but that's not how your character would act in this situation.
#Person2#: But Jason and Laura have been together for three years. Don't you think his reaction would be one of both anger and sadness?
#Person1#: At this point, no. I think he would react the way most guys would, and then later on, we would see his real feelings.
#Person2#: I'm not so sure about that.
#Person1#: Let's try it my way, and you can see how you feel when you're saying your lines. After that, if it still doesn't feel right, we can try something else.
        
        What conversation is going on?
        #Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that M

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION:
#Person1: I'm thinking of upgrading my computer.


## Step3: Few Shot

In [69]:
example_indices_full = [40, 100]
example_index_to_summarize = 200

few_shot_prompt = make_few_shot_prompt(example_indices_full, example_index_to_summarize, "What conversation is going on?")

print(few_shot_prompt)

generate_summary(example_index_to_summarize, few_shot_prompt)


        Dialogue:
        
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
        
        What conversation is going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        
        
        Dialogue:
        
        #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Per

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION:
#Person1: Have you considered upgrading your system? #Person2: Yes, but I'm not sure what exactly I would need. #Person1: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising. #Person2: You might also want to upgrade your hardware because it is pretty outdated now. #Person1: You'd probably need a faster processor,


In [70]:
example_indices_full = [40, 100, 150]
example_index_to_summarize = 200

few_shot_prompt = make_few_shot_prompt(example_indices_full, example_index_to_summarize, "What conversation is going on?")

print(few_shot_prompt)

generate_summary(example_index_to_summarize, few_shot_prompt)


        Dialogue:
        
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
        
        What conversation is going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        
        
        Dialogue:
        
        #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Per

## Generative Configuration Parameters for Inference

In [71]:
def generate_summary(example_index_to_summarize, prompt, generation_config):
    summary = dataset['test'][example_index_to_summarize]['summary']
    
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    model_output = model.generate(inputs=inputs, generation_config=generation_config)
    output = tokenizer.decode(token_ids=model_output[0], skip_special_tokens=True)

    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION:\n{output}')

In [ ]:
example_indices_full = [40, 100, 150]
example_index_to_summarize = 200

few_shot_prompt = make_few_shot_prompt(example_indices_full, example_index_to_summarize, "What conversation is going on?")

In [75]:
generation_config = GenerationConfig(max_new_tokens=50)

generate_summary(example_index_to_summarize, few_shot_prompt, generation_config)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION:
#Person1: Have you considered upgrading your system? #Person2: Yes, but I'm not sure what exactly I would need. #Person1: You could consider adding a painting program to your software. It would


In [76]:
generation_config = GenerationConfig(max_new_tokens=10)

generate_summary(example_index_to_summarize, few_shot_prompt, generation_config)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION:
#Person1: Have you considered upgrading your


In [77]:
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)

generate_summary(example_index_to_summarize, few_shot_prompt, generation_config)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION:
#Person1: Have you considered upgrading your system? #Person2: Yes, but I'm not sure what exactly I would need. #Person1: You could consider adding a painting program to your software. It would


In [78]:
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1, top_k=25)

generate_summary(example_index_to_summarize, few_shot_prompt, generation_config)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION:
#Person1: Have you considered upgrading your system? #Person2: Yes, but I'm not sure what exactly I would need. #Person1: You could consider adding a painting program to your software. It would


In [79]:
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1, top_p=0.25)

generate_summary(example_index_to_summarize, few_shot_prompt, generation_config)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION:
#Person1: Have you considered upgrading your system? #Person2: Yes, but I'm not sure what exactly I would need. #Person1: You could consider adding a painting program to your software. It would


In [83]:
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1, top_p=0.50, top_k=100)

generate_summary(example_index_to_summarize, few_shot_prompt, generation_config)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION:
#Person1: Have you considered upgrading your system? #Person2: Yes, but I'm not sure what exactly I would need. #Person1: You could consider adding a painting program to your software. It would
